# AML kaggle competition

### Importing packages

In [30]:
#import packages
import datetime
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
import sklearn.metrics as metrics
from sklearn.neighbors import RadiusNeighborsClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, r2_score


import seaborn as sns

from pandas.tseries.holiday import USFederalHolidayCalendar as calendar


from keras.models import Sequential
from keras.layers import Dense
np.random.seed(7)
from keras.models import Sequential
from keras.layers import Dense



# Collect Datafiles

In [31]:
#import data files
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
Merged= df_train.append(df_test)

df_airports = pd.read_csv('airports.csv')
df_airports = df_airports.drop(columns='name')

df_weather = pd.read_csv('weather.csv')

# df_train= df_train[df_train['carrier']=='AA']
df=df_train

airDict=df_airports[['faa','tz','dst']].set_index('faa').to_dict()

DLat = df_airports[['faa','lat']].groupby(['faa']).mean().to_dict()['lat']
DLon = df_airports[['faa','lon']].groupby(['faa']).mean().to_dict()['lon']
thatday=df_train[['month','day','is_delayed']].groupby(['month','day']).mean().to_dict()['is_delayed']

In [32]:
def weekday(date):
    return datetime.datetime(date[0], date[1], date[2]).weekday()

In [33]:


df_weather['wind_speed'] = df_weather[['wind_speed']].fillna(df_weather.wind_speed.mean())
df_weather['visib'] = df_weather[['visib']].fillna(df_weather.visib.mean())

delaycarrier = df_train[['is_delayed','carrier']].groupby('carrier').mean().to_dict()['is_delayed']
delayODC = df_train[['is_delayed','origin','dest','carrier']].groupby(['origin','dest','carrier']).mean().to_dict()['is_delayed']


originvisibility = df_weather[['origin','month','day','hour','visib']].groupby(['origin','month','day','hour']).mean().to_dict()['visib']
originwind = df_weather[['origin','month','day','hour','wind_speed']].groupby(['origin','month','day','hour']).mean().to_dict()['wind_speed']
originpressure = df_weather[['origin','month','day','hour','pressure']].groupby(['origin','month','day','hour']).mean().to_dict()['pressure']
originhumidity = df_weather[['origin','month','day','hour','humid']].groupby(['origin','month','day','hour']).mean().to_dict()['humid']
originprecip = df_weather[['origin','month','day','hour','precip']].groupby(['origin','month','day','hour']).mean().to_dict()['precip']
origindir = df_weather[['origin','month','day','hour','wind_dir']].groupby(['origin','month','day','hour']).mean().to_dict()['wind_dir']
origintemp = df_weather[['origin','month','day','hour','temp']].groupby(['origin','month','day','hour']).mean().to_dict()['temp']
origindew = df_weather[['origin','month','day','hour','dewp']].groupby(['origin','month','day','hour']).mean().to_dict()['dewp']
originwindgust = df_weather[['origin','month','day','hour','wind_gust']].groupby(['origin','month','day','hour']).mean().to_dict()['wind_gust']



tempdew = df_weather['dewp'].mean()
tempmean = df_weather['temp'].mean()
windmean = df_weather['wind_speed'].mean()
visibmean = df_weather['visib'].mean()
pressuremean = df_weather['pressure'].mean()
humiditymean = df_weather['humid'].mean()
precipmean = df_weather['precip'].mean()
dirmean= df_weather['wind_dir'].mean()

In [34]:
def prepare(df):
    
    
#     df['hour']=[int(row[1]/100) for row in df[['sched_dep_time']].itertuples()]
#     df['minute']=[row[1]-int(row[1]/100)*100 for row in df[['sched_dep_time']].itertuples()]
#     df['DateTime'] = [datetime.datetime(row[1], row[2], row[3],row[4],row[5]) for row in df[['year','month','day','hour','minute']].itertuples()]
    
    df['sched_dep_hour'] = [int(i) for i in round(df['sched_dep_time']/100)]
    df['sched_arr_hour'] = [int(i) for i in round(df['sched_arr_time']/100)]
    
    


    df['sin_day'] = np.sin(2*np.pi*df.day/31)
    df['cos_day'] = np.cos(2*np.pi*df.day/31)

    df['sin_month'] = np.sin(2*np.pi*df.month/12)
    df['cos_month'] = np.cos(2*np.pi*df.month/12)

    df['sched_dep_hour_sin'] = np.sin(2*np.pi*df.sched_dep_hour/23)
    df['sched_dep_hour_cos'] = np.cos(2*np.pi*df.sched_dep_hour/23)

    df['sched_arr_hour_sin'] = np.sin(2*np.pi*df.sched_arr_hour/23)
    df['sched_arr_hour_cos'] = np.cos(2*np.pi*df.sched_arr_hour/23)

    distlist = df['distance']
    df['distance'] = distlist/max(distlist)
    
    df['delayODC'] = [delayODC.get(tuple(row)[1:],0.5) for row in df[['origin','dest','carrier']].itertuples()]
    
    df['thatday'] = [thatday.get(tuple(row)[1:]) for row in df[['month','day']].itertuples()]

    df['weekday'] = [weekday(tuple(row)[1:]) for row in df[['year','month','day']].itertuples()]
    df['weekdaysin'] = np.sin(2*np.pi*df.weekday/6)
    df['weekdaycos'] = np.cos(2*np.pi*df.weekday/6)
    
    df['originwind_gust'] = [originhumidity.get(tuple(row)[1:]) for row in df[['origin','month','day','sched_dep_hour']].itertuples()]
    df['originwind_gust'] = df['originwind_gust'].fillna(0)
    df['originwind_gust'] =  df['originwind_gust']/ df['originwind_gust'].max()
    
    df['Dlat'] = [DLat.get(d) for d in df['dest']]
    df['Dlon'] = [DLon.get(d) for d in df['dest']]
    
    df['Dlat'] = df['Dlat'].fillna(df['Dlat'].mean())
    df['Dlon'] = df['Dlon'].fillna(df['Dlon'].mean())
    
    df['Dlat'] = df['Dlat']/max(df['Dlat'])
    df['Dlon'] = df['Dlon']/max(df['Dlon'])

    
    df['originvis'] = [originvisibility.get(tuple(row)[1:],visibmean) for row in df[['origin','month','day','sched_dep_hour']].itertuples()]
    df['originvis'] =  df['originvis']/ df['originvis'].max()
    
    df['originwind'] = [originwind.get(tuple(row)[1:],windmean) for row in df[['origin','month','day','sched_dep_hour']].itertuples()]
    df['originwind'] =  df['originwind']/ df['originwind'].max()
    
    df['originhumidity'] = [originhumidity.get(tuple(row)[1:],humiditymean) for row in df[['origin','month','day','sched_dep_hour']].itertuples()]
    df['originhumidity'] = df['originhumidity'].fillna(humiditymean)
    df['originhumidity'] =  df['originhumidity']/ df['originhumidity'].max()
    
    df['originpressure'] = [originpressure.get(tuple(row)[1:],pressuremean) for row in df[['origin','month','day','sched_dep_hour']].itertuples()]
    df['originpressure'] = df['originpressure'].fillna(pressuremean)
    df['originpressure'] =  df['originpressure']/ df['originpressure'].max()
    
    df['originprecip'] = [originprecip.get(tuple(row)[1:],precipmean) for row in df[['origin','month','day','sched_dep_hour']].itertuples()]
    df['originprecip'] =  df['originprecip']/ df['originprecip'].max()
    
    df['origindir'] = [origindir.get(tuple(row)[1:],dirmean) for row in df[['origin','month','day','sched_dep_hour']].itertuples()]
    df['origindir'] = df['origindir'].fillna(dirmean)
    df['origindir_sin'] = np.sin(2*np.pi*df.origindir/360)
    df['origindir_cos'] = np.cos(2*np.pi*df.origindir/360)
    
    df['origintemp'] = [origintemp.get(tuple(row)[1:],tempmean) for row in df[['origin','month','day','sched_dep_hour']].itertuples()]
    df['origintemp'] = df['origintemp'].fillna(tempmean)
    df['origintemp'] =  df['origintemp']/ df['origintemp'].max()
    
    df['origindew'] = [origindew.get(tuple(row)[1:],tempmean) for row in df[['origin','month','day','sched_dep_hour']].itertuples()]
    df['origindew'] = df['origindew'].fillna(tempmean)
    df['origindew'] =  df['origindew']/ df['origindew'].max()
    
#     df = df.drop(columns=['dest','date','weekday','carrier','month','day','year','id','sched_dep_time','sched_arr_time','sched_dep_hour','sched_arr_hour','originwind'])
    df = df.drop(columns=['origindir','month','day','year','sched_dep_time','sched_arr_time','sched_arr_hour'])
    df = pd.get_dummies(df)


    return df

In [35]:
merged = prepare(Merged)

train=merged.head(len(df_train))
smalltrain = train.head(int(len(train)*0.8))
validation = train.tail(int(len(train)*0.2))

test=merged.tail(len(df_test))

In [36]:
y_validation, x_validation = validation['is_delayed'], validation.drop(columns='is_delayed')
y_smalltrain, x_smalltrain = smalltrain['is_delayed'], smalltrain.drop(columns='is_delayed')
y_train, x_train = train['is_delayed'], train.drop(columns='is_delayed')
x_test = test

# TreeNN Model

## Train Trees on Training Data

In [37]:
RF_All = RandomForestClassifier(n_estimators=500,max_features=15,random_state=1)
RF_Date = RandomForestClassifier(n_estimators=500,random_state=1)
RF_Weather = RandomForestClassifier(n_estimators=500,random_state=1)
RF_ODC = RandomForestClassifier(n_estimators=500,max_features=50,random_state=1)




weather=['origintemp','origindew','distance']
dateandtime=['sin_day','cos_day','sin_month','cos_month','sched_dep_hour_sin','sched_dep_hour_cos','weekday','thatday']
O=[col for col in x_smalltrain if col.startswith('origin_')]
D=[col for col in x_smalltrain if col.startswith('dest_')]
C=[col for col in x_smalltrain if col.startswith('carrier_')]
LONLAT=['Dlon','Dlat']

all_smalltrain=x_smalltrain[O+D+C+weather+dateandtime]
all_val=x_validation[O+D+C+weather+dateandtime]
all_test=x_test[O+D+C+weather+dateandtime]

date_smalltrain=x_smalltrain[dateandtime]
date_val=x_validation[dateandtime]
date_test=x_test[dateandtime]

weather_smalltrain=x_smalltrain[weather]
weather_val=x_validation[weather]
weather_test=x_test[weather]

ODC_smalltrain=x_smalltrain[O+D+C+['distance']]
ODC_val=x_validation[O+D+C+['distance']]
ODC_test=x_test[O+D+C+['distance']]


RF_All.fit(all_smalltrain, y_smalltrain)
print('done')
RF_Date.fit(date_smalltrain, y_smalltrain)
print('done')
RF_Weather.fit(weather_smalltrain, y_smalltrain)
print('done')
RF_ODC.fit(ODC_smalltrain, y_smalltrain)
print('done')

done
done
done
done


## Trees predict the training data and NN is trained to combine predictions

In [38]:
treeResults=pd.DataFrame()
i=1
for RF in [[RF_ODC,ODC_val],[RF_Weather,weather_val],[RF_Date,date_val],[RF_All,all_val]]:
    for tree in RF[0].estimators_:
        treeResults[str(i)]=tree.predict(RF[1])
        i+=1
    #     print(i,  metrics.roc_auc_score(y_validation,treeResults[str(i)]))


    

In [39]:
model = Sequential()
model.add(Dense(10, input_dim=2000, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(treeResults, y_validation, epochs=3, batch_size=100)


Epoch 1/3
33714/33714 [==============================] - 5s 152us/step - loss: 0.5835 - acc: 0.6975
Epoch 2/3
33714/33714 [==============================] - 4s 131us/step - loss: 0.5460 - acc: 0.7257
Epoch 3/3
33714/33714 [==============================] - 5s 150us/step - loss: 0.5397 - acc: 0.7288


## Trees predict the validation data and NN is used to combine predictions

In [40]:
treeResults_test=pd.DataFrame()
i=1
for RF in [[RF_ODC,ODC_test],[RF_Weather,weather_test],[RF_Date,date_test],[RF_All,all_test]]:
    for tree in RF[0].estimators_:
        treeResults_test[str(i)]=tree.predict(RF[1])
        i+=1
    #     print(i,  metrics.roc_auc_score(y_validation,treeResults[str(i)]))






predictions = model.predict(treeResults_test)
predictions=[i[0] for i in predictions]


In [42]:
submission = pd.DataFrame()
submission["id"] = test["id"]
submission["is_delayed"] = predictions
submission.to_csv("submission_maybe78.csv", index=False)

In [ ]:
0.78 AUC with 500*4 trees
weather=['origintemp','origindew','distance']
dateandtime=['sin_day','cos_day','sin_month','cos_month','sched_dep_hour_sin','sched_dep_hour_cos','weekday','thatday']